# CNN Model

This code is an adaptation of the CNN model provided by Professor Jorge Henriques in the course slides, the CNN model developed for another course (available at https://github.com/leosousa200/RetinopathyAlgorithm), a guide to multi-input CNN provided by Keras (available at https://keras.io/guides/functional_api/) and a CNN model available on Kaggle (at https://www.kaggle.com/code/vishwasgpai/guide-for-creating-cnn-model-using-csv-file/notebook).

### Imports

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout

from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

### Import and Organize Data

In [22]:
numeric = pd.read_csv("COVID_numerics.csv").values
images  = pd.read_csv("COVID_IMG.csv",header=None).values # avoids ignoring first line!

X_num = numeric[:,0:8] # 8 columns (GENDER,AGE,MARITAL STATUS,VACINATION,RESPIRATION CLASS,HEART RATE,SYSTOLIC BLOOD PRESSURE,TEMPERATURE)
T = numeric[:,8] # Target
X_img = images[:,:] # each line represents a 21x21 image

### Reshape Image Data (21x21)

In [23]:
X_img = X_img.reshape(X_img.shape[0],21,21,1) # resize images into 21x21x1

### Train-Test Split

In [25]:
# We need to associate each X_img entry to each X_num entry
# To do so, we can stack them, so each entry has now 8+(21*21) columns
# This is important for the train-test split, so we can assure both sets use the same entries!
# Check https://www.geeksforgeeks.org/numpy-hstack-in-python/

#X = np.hstack([X_num,X_img.reshape(X_img.shape[0],21*21)])

#X_train, X_test, T_train, T_test = train_test_split(X, T, test_size=0.3, random_state=42)

X_img_train, X_img_test, X_num_train, X_num_test, T_train, T_test = train_test_split(X_img, X_num, T, test_size=0.3, random_state=42)

#X_train = np.hstack([X_num_train, X_img_train.reshape(X_img_train.shape[0],21*21)])
#X_test = np.hstack([X_num_test, X_img_test.reshape(X_img_test.shape[0],21*21)])

# One-hot Encoding
#T_train = to_categorical(T_train)
#T_test = to_categorical(T_test)

### Model

In [26]:
numeric_input = keras.Input(shape=(8,))
image_input = keras.Input(shape=(21,21,1))

# CNN Path Only for the Images
x_img = Conv2D(64, (3, 3), activation='relu')(image_input)
x_img = MaxPooling2D(pool_size=(2, 2))(x_img)
x_img = Conv2D(32, (3, 3), activation='relu')(x_img)
x_img = MaxPooling2D(pool_size=(2, 2))(x_img)
x_img = Flatten()(x_img)

# MLP Path Only for Numerical Data
x_num = Dense(64, activation='relu')(numeric_input)

# Concatenate Outputs
concatenated = np.concatenate([x_img, x_num])

# Final Output Dense Layer
output = Dense(1,activation='sigmoid')(concatenated) # this is a binary classification problem, so we can use the Sigmoid activation function

model = Model(inputs=[image_input,numeric_input], outputs=output)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


c:\Users\User\Desktop\Projeto AC\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Train Model

In [ ]:
history = model.fit([X_img_train, X_num_train], T_train, epochs=12, batch_size=64)
print("Loss: ", history.history['loss'])

train = model.predict([X_img_train, X_num_train]).round()

Epoch 1/12


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(None, 449), dtype=float32). Expected shape (None, 21, 21, 1), but input has incompatible shape (None, 449)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 449), dtype=float32)
  • training=True
  • mask=None

### Test Model

In [ ]:
Ytest = model.predict(X_test).round()
scores = model.evaluate(X_test, T_test, verbose=0)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

### Evaluate Model

In [ ]:
cm = confusion_matrix(T_test, Ytest)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

print(classification_report(T_test,Ytest))